# Auxiliar 3


## 📚 Objetivos de la clase 📚

La clase auxiliar de esta semana tendrá varios objetivos:

- Explicar el problema de la representación Bag of Words.
- Motivación y repaso de qué son los Word Embeddings.
- Explicación de Word2Vec.
- Entrenar nuestros propios `word embeddings`.
- Utilizaremos estos embeddings pre-entrenados para mejorar la capacidad de nuestros modelos en tareas nuevas y en la resuelta en el auxiliar pasado.

Una vez resuelto, pueden utilizar cualquier parte del código que les parezca prudente para la Tarea 3 😊.

## **Motivación**

Partamos por decir que una red neuronal no es más que una serie de operaciones matemáticas sobre vectores con una gran cantidad de dimensiones (tensores). Por ende, si queremos entrenar un modelo necesitamos transformar el texto original a vectores numéricos.

Una de las soluciones más simples a este problema es la representación de Bag of Words (BoW). Si aplicamos este método a cada palabra de cada documento, tendremos un vector one hot encoding por cada palabra. Esto quiere decir que tendremos vectores del largo del vocabulario $V$, con un 1 en la posición asociada a la palabra representada.

Y estamos listos? Podemos entrenar redes neuronales?

La verdad es que no es así, y es que estamos ignorando un gran problema con este enfoque. 😞


### El gran problema de Bag of Words

Pensemos en estas 3 frases como documentos:

- $doc_1$: `¡Buenísima la marraqueta!`
- $doc_2$: `¡Estuvo espectacular ese pan francés!`
- $doc_3$: `!Buenísima esa pintura!`

Sabemos $doc_1$ y $doc_2$ hablan de lo mismo 🍞🍞👌 y que $doc_3$ 🎨 no tiene mucho que ver con los otros.

Supongamos que queremos ver que tan similares son ambos documentos.
Para esto, generamos un modelo `Bag of Words` sobre el documento, aplicando este método por cada palabra para luego tener la representación final.


Es decir, transformamos cada palabra a un vector one-hot y luego los sumamos por documento.

Por simplicidad, omitiremos algunas stopwords y consideramos pan frances como un solo token. Así nos quedaría el siguiente vocabulario:

$$v = \{buenísima, marraqueta, estuvo, espectacular, pan\ francés, pintura\}$$

Entonces, el $\vec{doc_1}$ quedará:

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} =
  \begin{bmatrix}1 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix}$$

El $\vec{doc_2}$ quedará:

$$\begin{bmatrix}0 \\ 0 \\ 1 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1\\ 0\end{bmatrix} =
  \begin{bmatrix}0 \\ 0 \\ 1 \\ 1 \\ 1\\ 0\end{bmatrix}$$

Y el $\vec{doc_3}$:

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix} =
  \begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix}$$



**¿Cuál es el problema?**

`buenísima` $\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `espectacular` $ \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0\end{bmatrix}$ representan ideas muy similares. Por otra parte, sabemos que `marraqueta` $\begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `pan francés` $\begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1 \\0\end{bmatrix}$ se refieren al mismo objeto. Pero en este modelo, estos **son totalmente distintos**. Es decir, los vectores de las palabras que `buenísima` y `espectacular` son tan distintas como `marraqueta` y `pan francés`. Esto se debe a que cada palabra ocupa una dimensión distinta a las demás y son completamente independientes. Esto evidentemente, repercute en la calidad de los modelos que creamos a partir de nuestro Bag of Words.

![BoW](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/BoW-Problem.png)



Ahora, si queremos ver que documento es mas similar a otro usando distancia euclidiana, veremos que:

$$d(doc_1, doc_2) = 2.236$$
$$d(doc_1, doc_3) = 1.414$$

Es decir, $doc_1$ se parece mas a $doc_3$ aunque nosotros sabemos que $doc_1$ y $doc_2$ nos están diciendo lo mismo!


Nos gustaría que eso no sucediera. Que existiera algún método que nos permitiera hacer que palabras similares tengan representaciones similares. Y que con estas, representemos mejor a los documentos, sin asumir que en el espacio son geométricamente equidistantes, ya que esto no es verdad en la vida real.


--------------------

## **Hipótesis Distribucional**

Estamos buscando algún enfoque que nos permita representar las palabras de forma no aislada, de manera tal que además capture el significado de esta.

Pensemos un poco en la **hipótesis distribucional**. Esta plantea que:

    "Palabras que ocurren en contextos iguales tienden a tener significados similares."

O equivalentemente,

    "Una palabra es caracterizada por la compañía que esta lleva."

Esto nos puede hacer pensar que podríamos usar los contextos de las palabras para generar vectores que describan mejor dichas palabras: en otras palabras, los `Distributional Vectors`.

Por ejemplo, complete la siguiente frase:

Pintaré la muralla de mi casa de color _____

Puede ser rojo, blanco, mostaza, etc..

Son palabras que a uno se les viene a la mente sólo mirando el contexto entregado, por ende podríamos decir que esas son palabras similares, o al menos muy distintas a Murciélago.


### Opción 1: Word-Context Matrix

Es una matriz donde cada celda $(i,j)$ representa la co-ocurrencia entre una palabra objetivo/centro $w_i$ y un contexto $c_j$. El contexto son las palabras dentro de ventana de tamaño $k$ que rodean la palabra central.

Cada fila representa a una palabra a través de su contexto. Como pueden ver, ya no es un vector one-hot, si no que ahora contiene mayor información.

El tamaño de la matriz es el tamaño del vocabulario $V$ al cuadrado. Es decir $|V|*|V|$.

<img src="https://raw.githubusercontent.com/dccuchile/CC6205/master/slides/pics/distributionalSocher.png" alt="Word-context matrices" style="width: 400px;"/>


**Problema: Creada a partir de un corpus respetable, es gigantezca**.

Por ejemplo, para $|v| = 100.000$, la matriz tendrá $\frac{100000 * 100000 * 4}{10^9} = 40gb $. (Recordando que un entero ocupara 4 bytes)

- Es caro mantenerla en memoria.
- Los clasificadores no funcionan tan bien con tantas dimensiones (ver [maldición de la dimensionalidad](https://es.wikipedia.org/wiki/Maldici%C3%B3n_de_la_dimensi%C3%B3n)).

**¿Habrá una mejor solución?**

---------------------

### Opción 2: **Word Embeddings**

Es una de las representaciones más populares del vocabulario de un corpus. La idea principal de los Word Embeddings es crear representaciones vectoriales densas y de baja dimensionalidad $(d << |V|)$ de las palabras a partir de su contexto.

Volvamos a nuestro ejemplo anterior: `buenísima` y `espectacular` ocurren muchas veces en el mismo contexto, por lo que los embeddings que los representan debiesen ser muy similares... (*ejemplos de mentira hechos a mano*):

`buenísima` $\begin{bmatrix}0.32 \\ 0.44 \\ 0.92 \\ .001 \end{bmatrix}$ y `espectacular` $\begin{bmatrix}0.30 \\ 0.50 \\ 0.92 \\ .002 \end{bmatrix}$ versus `marraqueta`  $\begin{bmatrix}0.77 \\ 0.99 \\ 0.004 \\ .1 \end{bmatrix}$ el cuál es claramente distinto.


Pero, ¿Cuál es la utilidad de de crear estos vectores en NLP o en el área de Machine Learning en general?

Supongamos que tienen una enfermedad grave y deben ser operados el día de mañana. Le dan a elegir entre ser operados por un estudiante de primer año de medicina con algo de conocimiento médico o bien ser operados por un niño de 5 años 👶. ¿A quién elegirías?

Espero que tu opción haya sido el estudiante con una pequeña noción de los términos médicos implicados en una intervención así. Algo así es lo que se quiso lograr en el [paper](https://arxiv.org/abs/1301.3781) presentado por Mikolov en 2013, aludiendo a la herramienta **Word2Vec**. La idea es que si quieres resolver por ejemplo una tarea de clasificación de texto, ¿no sería útil utilizar el conocimiento de algún modelo pre-entrenado en una tarea similar de texto?. Claro, sería útil partir con los pesos entrenados por otra red, realizando lo que se llama **transfer learning**.

Ya pero.. ¿Cómo generamos estos vectores? ¿Cómo podemos capturar el contexto? ¿Cuál sería esa task auxiliar a utilizar?



##### **Word2vec y Skip-gram**

Word2Vec es probablemente el paquete de software mas famoso para crear word embeddings utilizando distintos modelos que emplean redes neuronales *shallow* o poco profundas.

Este nos provee herramientas para crear distintos tipos de modelos, tales como `Skip-Gram` y `Continuous Bag of Word (CBOW)`. En este caso, solo veremos `Skip-Gram`.

**Skip-gram** es una task auxiliar con la que crearemos nuestros embeddings. Esta tarea involucra tanto a las palabras y al contexto de ellas. Consiste en que por cada palabra del dataset, debemos predecir las palabras de su contexto (las palabras presentes en ventana de algún tamaño $k$).

![Overview](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/overview-skipgram.png)

Para resolverla, usaremos una red de una sola capa oculta. Los pesos ya entrenados de esta capa serán los que usaremos como embeddings.

#### Detalles del Modelo

- Como dijimos, el modelo será una red de una sola capa. La capa oculta tendrá una dimensión $d$ la cual nosotros determinaremos. Esta capa no tendrá función de activación. Sin embargo, la de salida si, la cual será una softmax para obtener las distribuciones de probabilidades y así ver cuáles palabras pertenecen o no al contexto.

- El vector de entrada, de tamaño $|V|$, será un vector one-hot de la palabra que estemos viendo en ese momento.

- La salida, también de tamaño $|V|$, será un vector que contenga la distribución de probabilidad de que cada palabra del vocabulario pertenezca al contexto de la palabra de entrada.

- Al entrenar, se comparará la distribución de los contextos con la suma de los vectores one-hot del contexto real.


(marraqueta, Estuvo), (marraqueta, buenisima), (marraqueta, la)
![Skip Gram](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/Skip-gram.png)


Nota: Esto es computacionalmente una locura. Por cada palabra de entrada, debemos calcular la probabilidad de aparición de todas las otras. Imaginen el caso de un vocabulario de 100.000 de palabras y de 10000000 oraciones...

La solución a esto es modificar la task a *Negative Sampling*. Esta transforma este problema de $|V|$ clases a uno binario. Sin embargo, no lo veremos por el tiempo, pero están muy bien explicado en el [video de la cátedra](https://www.youtube.com/watch?v=XDxzQ7JU95U&feature=youtu.be).


### La capa Oculta y los Embeddings

Al terminar el entrenamiento, ¿Qué nos queda en la capa oculta?

Una matriz de $v$ filas por $d$ columnas, la cual contiene lo que buscabamos: Una representación continua de todas las palabras de nuestro vocabulario.  

**Cada fila de la matriz es un vector que contiene la representación continua una palabra del vocabulario.**


<img src="http://mccormickml.com/assets/word2vec/word2vec_weight_matrix_lookup_table.png" alt="Capa Oculta 1" style="width: 400px;"/>

¿Cómo la usamos eficientemente?

Simple: usamos los mismos vectores one-hot de la entrada y las multiplicamos por la matriz:

<img src="http://mccormickml.com/assets/word2vec/matrix_mult_w_one_hot.png" alt="Skip Gram" style="width: 400px;"/>

### Visualización

Veamos cómo se ven los embeddings de Word2Vec entrenados sobre un corpus gigante en Inglés. Para facilitar el análisis se reducen las 200 dimensiones a 3. El link a la visualización es el siguiente: Visualización: https://projector.tensorflow.org/

### Espacio multidimensional

Teniendo nuestro embeddings entonces podríamos hacer operaciones tan interesantes como las siguientes:

Manzana + Púrpura -> Ciruela

Rey - Hombre + Mujer -> Reina

Si bien no es posible obtener exactamente dichos vectores, esperaríamos que las palabras más cercanas al vector resultante serían las entregadas, obteniendo así un significado de las palabras según su contexto.


### Fuentes

Word2vec:
- mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
- https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa

Gensim:
- https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

Nota: Las últimas 2 imagenes pertenecen a [Chris McCormick](http://mccormickml.com/about/)


In [ ]:
# Contextualized word embeddings: BERT, ELMO, FLAIR.

x = 'El banco estaba lleno.'
y = 'El banco de sangre necesita personal.'

## **Entrenar nuestros Embeddings**

Para entrenar nuestros embeddings, usaremos el paquete gensim. Este trae una muy buena implementación de `word2vec`.




In [ ]:
import re
import pandas as pd
from time import time
from collections import defaultdict
import string
import multiprocessing
import os
import requests
import numpy as np

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

# scikit-learn
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin

# visualizaciones
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

### Cargar el dataset y limpiar

Nota: Pandas descomprime por si mismo el archivo bz2. Pueden descomprimirlo manualmente usando 7zip.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# descargamos el dataset completo (~40mb)
dataset = pd.read_json(
    'https://github.com/dccuchile/CC6205/releases/download/Data/biobio_clean.bz2',
    encoding="utf-8")

dataset_r = dataset.copy(deep = True)

In [ ]:
dataset.head(10)

,author,author_link,title,link,category,subcategory,content,tags,embedded_links,publication_datetime
0,Yerko Roa,/lista/autores/yroa,Colapsa otro segmento de casa que se derrumbó ...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,Noticia en Desarrollo Estamos recopilando m...,[],[],1565778000000
1,Valentina González,/lista/autores/vgonzalez,Policía busca a mujer acusada de matar a su pa...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Detectives de la Policía de Investigaciones ...,"[#parricidio, #PDI, #Pudahuel, #Región Metropo...",[https://media.biobiochile.cl/wp-content/uploa...,1565771820000
2,Felipe Delgado,/lista/autores/fdelgado,Dos detenidos en Liceo de Aplicación: protagon...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Dos detenidos fue el saldo de una serie de i...,"[#Incendio, #Liceo de Aplicación, #Región Metr...",[],1565772480000
3,Matías Vega,/lista/autores/mvega,Apoyo transversal: Senado aprueba en general p...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,La sala del Senado aprobó en general el proy...,"[#Inmigración, #Inmigrantes, #Ley, #Migración,...",[https://media.biobiochile.cl/wp-content/uploa...,1565772720000
4,Valentina González,/lista/autores/vgonzalez,Evacuación espontánea en Instituto Nacional po...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,La mañana de este miércoles se produjo una e...,"[#Carabineros, #FFEE, #Gases Lacrimógenos, #In...",[],1565772960000
5,Gonzalo Cifuentes,/lista/autores/gcifuentes,Alcalde Sharp lamenta mortal derrumbe y afirma...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,"El alcalde de Valparaíso, Jorge Sharp , se r...","[#Alcalde, #derrumbe, #derrumbe en valparaíso,...",[],1565773080000
6,Catalina Díaz,/lista/autores/catalinadiaz,Joven resulta grave tras ser apuñalado en even...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-los-lagos,"Un joven se encuentra en estado grave, tras ...","[#Agresión, #Apuñalado, #Carabineros, #Chiloé,...",[https://media.biobiochile.cl/wp-content/uploa...,1565773140000
7,Matías Vega,/lista/autores/mvega,En prisión preventiva queda acusado de violar ...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-del-maule,Un caso criminal -que era investigado de man...,"[#Deficiencia mental, #Región del Maule, #Talc...",[],1565774520000
8,Emilio Lara,/lista/autores/elara,Cualquier chileno puede ser objeto de escuchas...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,"Bajo carácter privado, como dictamina la Ley...","[#Ejército, #escuchas telefónicas, #Espionaje,...",[https://media.biobiochile.cl/wp-content/uploa...,1565774820000
9,Valentina González,/lista/autores/vgonzalez,Nuevos pórticos y reducción de espera en semáf...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,El alcalde Joaquín Lavín anunció medidas lue...,"[#asaltos a conductores, #encerronas, #las Con...",[],1565776560000


In [ ]:
# unir titulo con contenido de la noticia
content = dataset['title'] + dataset['content']

In [ ]:
content.head()

0    Colapsa otro segmento de casa que se derrumbó ...
1    Policía busca a mujer acusada de matar a su pa...
2    Dos detenidos en Liceo de Aplicación: protagon...
3    Apoyo transversal: Senado aprueba en general p...
4    Evacuación espontánea en Instituto Nacional po...
dtype: object

In [ ]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [ ]:
content.shape

(26413,)

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
from collections import Counter

# limpiar puntuaciones y separar por tokens.
punctuation = string.punctuation + "«»“”‘’…—"
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())

def simple_tokenizer(doc, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()
    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc

cleaned_content = [simple_tokenizer(doc) for doc in content.values]

In [ ]:
print("Ejemplo de alguna noticia: {}".format(cleaned_content[14]))

Ejemplo de alguna noticia: ['Ministra', 'Cubillos', 'extensión', 'paro', 'docente', 'problema', 'propuesta', 'ministra', 'Educación', 'Marcela', 'Cubillos', 'refirió', 'a', 'votación', 'realizada', 'interior', 'Colegio', 'Profesores', 'decidió', 'mantener', 'paro', 'pese', 'llamado', 'presidente', 'gremio', 'Mario', 'Aguilar', 'a', 'replegarse', 'entrevista', 'programa', 'Expreso', 'Bío', 'Bío', 'Radio', 'ministra', 'valoró', 'dichos', 'Aguilar', 'previo', 'a', 'votación', 'asegurando', 'efecto', 'positivo', 'vuelta', 'clases', 'Paro', 'profesores', 'extiende', 'semana', '255', 'votos', 'marcaron', 'diferencia', 'sufragio', '95', 'colegios', 'funcionan', 'normalidad', 'bajaron', '490', 'colegios', 'quedaban', 'paro', 'a', 'Ministerio', 'seguirá', 'negociando', 'ofrecerá', 'considerando', 'mayoría', 'votó', 'extender', 'movilización', 'ministra', 'limitó', 'a', 'esperarán', 'resuelva', 'diálogos', 'internos', 'llevando', 'a', 'cabo', 'interior', 'gremio', 'línea', 'sostuvo', 'decisión',

### Extracción de Frases

Para crear buenas representaciones, es necesario tambien encontrar conjuntos de palabras que por si solas no tengan mayor significado (como `nueva` y `york`), pero que juntas que representen ideas concretas (`nueva york`).

Para esto, usaremos el primer conjunto de herramientas de `gensim`: `Phrases` y `Phraser`.

In [ ]:
# Phrases recibe una lista de oraciones, y junta bigramas que estén al menos 100 veces repetidos
# como un único token. Detrás de esto hay un modelo estadístico basado en frecuencias, probabilidades, etc
# pero en términos simples ese es el resultado

phrases = Phrases(cleaned_content, min_count=100, progress_per=5000)

Ahora, usamos `Phraser` para re-tokenizamos el corpus con los bigramas encontrados. Es decir, juntamos los tokens separados que detectamos como frases.

In [ ]:
bigram = Phraser(phrases)
sentences = bigram[cleaned_content]

In [ ]:
# para ver como quedan las noticias retokenizadas, quitar comentario a la siguiente linea:
print(sentences[1])

['Policía', 'busca', 'a', 'mujer', 'acusada', 'matar', 'a', 'padre', 'discusión', 'venta', 'vivienda', 'Pudahuel', 'Detectives', 'Policía_Investigaciones', 'realizan', 'peritajes', 'detener', 'a', 'mujer', '45', 'años', 'presunta', 'responsabilidad', 'ataque', 'arma', 'cortante', 'padre', 'causó', 'muerte', 'comuna', 'Pudahuel', 'ocurrió', 'calle', 'Presidente', 'Truman', 'intersección', 'Teniente', 'Cruz', 'acorde', 'a', 'declaración', 'hijo', 'víctima', 'hermano', 'victimaria', 'sostuvieron', 'enfrentamiento', 'verbal', 'a', 'intensión', 'Hernan', 'Silva', 'Pérez', 'vender', 'casa', 'Negocio', 'causado', 'molestia', 'hija', 'Tania', 'Silva', 'discusión', 'acudido', 'a', 'cocina', 'vivienda', 'volver', 'cuchillo', 'apuñalar', 'a', 'padre', 'primeras', 'diligencias', 'realizaron', 'carabineros', '45º', 'comisaría', 'tomaron', 'declaración', 'único', 'testigo', 'crimen', 'interior', 'vivienda', 'capitán', 'Carlos', 'Lagos', 'principal', 'hipótesis', 'apunta', 'a', 'discusión', 'dinero',

### Definir el modelo



Primero, como es usual, creamos el modelo. En este caso, usaremos uno de los primero modelos de embeddings neuronales: `word2vec`

Algunos parámetros importantes:

- `min_count`: Ignora todas las palabras que tengan frecuencia menor a la indicada.
- `window` : Tamaño de la ventana. Usaremos 4.
- `size` : El tamaño de los embeddings que crearemos. Por lo general, el rendimiento sube cuando se usan mas dimensiones, pero después de 300 ya no se nota cambio. Ahora, usaremos solo 200.
- `workers`: Cantidad de CPU que serán utilizadas en el entrenamiento.

In [ ]:
biobio_w2v = Word2Vec(min_count=10,
                      window=4,
                      vector_size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())

In [ ]:
Word2Vec()

### Construir el vocabulario

Para esto, se creará un conjunto que contendrá (una sola vez) todas aquellas palabras que aparecen mas de `min_count` veces.

In [ ]:
biobio_w2v.build_vocab(sentences, progress_per=10000)

### Entrenar el Modelo

A continuación, entenaremos el modelo.
Los parámetros que usaremos serán:

- `total_examples`: Número de documentos.
- `epochs`: Número de veces que se iterará sobre el corpus.

Es recomendable que tengan instalado `cpython` antes de continuar. Aumenta bastante la velocidad de entrenamiento.


In [ ]:
t = time()
biobio_w2v.train(sentences, total_examples=biobio_w2v.corpus_count, epochs=5, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 2.86 mins


Ahora que terminamos de entrenar el modelo, le indicamos que no lo entrenaremos mas.
Esto nos permitirá ejecutar eficientemente las tareas que realizaremos.

In [ ]:
biobio_w2v.init_sims(replace=True)

<ipython-input-20-8d993e9d3e98>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  biobio_w2v.init_sims(replace=True)


###  Guardar y cargar el modelo

Para ahorrar tiempo, usaremos un modelo preentrenado.

In [ ]:
# Si entrenaste el modelo y lo quieres guardar, descomentar el siguiente bloque.
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')
biobio_w2v.save('./pretrained_models/biobio_w2v.model')


# cargar el modelo (si es que lo entrenaron desde local.)
biobio_w2v = KeyedVectors.load("./pretrained_models/biobio_w2v.model", mmap='r')


In [ ]:
# descargar el modelo desde github
def read_model_from_github(url):
    if not os.path.exists('./pretrained_models'):
        os.mkdir('./pretrained_models')

    r = requests.get(url)
    filename = url.split('/')[-1]
    with open('./pretrained_models/' + filename, 'wb') as f:
        f.write(r.content)
    return True


[
    read_model_from_github(file) for file in [
        'https://github.com/dccuchile/CC6205/releases/download/Data/biobio_w2v.model',
    ]
]
# cargar el modelo (si es que lo entrenaron desde local.)
biobio_w2v = KeyedVectors.load("./pretrained_models/biobio_w2v.model", mmap='r')


## **Tasks: Palabras mas similares y Analogías**

### **Palabras mas similares**

Tal como dijimos anteriormente, los embeddings son capaces de codificar toda la información contextual de las palabras en vectores.

Y como cualquier objeto matemático, estos pueden operados para encontrar ciertas propiedades. Tal es el caso de las  encontrar las palabras mas similares, lo que no es mas que encontrar los n vecinos mas cercanos del vector.  

In [ ]:
biobio_w2v.wv.most_similar(positive=["perro"])

[('gato', 0.7324815392494202),
 ('perrito', 0.7017662525177002),
 ('cachorro', 0.6726856231689453),
 ('canino', 0.6614428758621216),
 ('mascota', 0.6354753971099854),
 ('animal', 0.6341222524642944),
 ('gatito', 0.6259311437606812),
 ('felino', 0.622412919998169),
 ('perros', 0.6207762360572815),
 ('perra', 0.5834704041481018)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["Chile"])

[('Latinoamérica', 0.48114773631095886),
 ('país', 0.4791499078273773),
 ('exportadores', 0.4531676173210144),
 ('chileno', 0.4481710195541382),
 ('Crecimiento', 0.4425714910030365),
 ('posiciona', 0.44039905071258545),
 ('Perú', 0.43912073969841003),
 ('Bolivia', 0.4357107877731323),
 ('chilenas', 0.43546050786972046),
 ('chilena', 0.430389940738678)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["Bolsonaro"])

[('ultraderechista_Jair', 0.7692869305610657),
 ('excapitán_Ejército', 0.7673073410987854),
 ('Jair_Bolsonaro', 0.7401224374771118),
 ('Haddad', 0.6947491765022278),
 ('exmilitar', 0.6393497586250305),
 ('Brasil', 0.618027925491333),
 ('Brasilia', 0.6159982681274414),
 ('nostálgico', 0.6144426465034485),
 ('Fernando_Haddad', 0.606082558631897),
 ('Ultraderechista', 0.603746771812439)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["Trump"])

[('Casa_Blanca', 0.709988534450531),
 ('Donald_Trump', 0.7093803882598877),
 ('mandatario_estadounidense', 0.7046915292739868),
 ('inquilino', 0.678024411201477),
 ('administración_Trump', 0.6302800178527832),
 ('Bolton', 0.6201831102371216),
 ('Unidos', 0.6197685599327087),
 ('Washington', 0.6131006479263306),
 ('presidente_estadounidense', 0.6014897227287292),
 ('Unidos_Donald', 0.5976207852363586)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["Pizza"])

[('Hut', 0.9047655463218689),
 ('Telepizza', 0.8243373036384583),
 ('ATT', 0.7196983098983765),
 ('Linio', 0.6911829113960266),
 ('Homecenter', 0.6828977465629578),
 ('Unimarc', 0.6740168333053589),
 ('Natura', 0.6694700717926025),
 ('Nestlé', 0.6646097302436829),
 ('Coffee', 0.6622934937477112),
 ('Sodimac', 0.6596288681030273)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["pizza"])

[('hamburguesas', 0.6893606185913086),
 ('cafeterías', 0.6606759428977966),
 ('degustaciones', 0.6389386653900146),
 ('repartidor', 0.6389126777648926),
 ('pizzas', 0.6339085102081299),
 ('platos', 0.6290442943572998),
 ('arroz', 0.6215819716453552),
 ('hamburguesa', 0.6195734739303589),
 ('vodka', 0.6181467771530151),
 ('chocolates', 0.616828441619873)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["Uber"])

[('Cabify', 0.7779495716094971),
 ('Eats', 0.700801432132721),
 ('DiDi', 0.676181435585022),
 ('Rappi', 0.6054359078407288),
 ('Conductores', 0.5858488082885742),
 ('aplicaciones', 0.5700352787971497),
 ('Beat', 0.566635251045227),
 ('app', 0.5640519857406616),
 ('choferes', 0.5415353178977966),
 ('taxistas', 0.5351073145866394)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["Huawei"])

[('ZTE', 0.7025798559188843),
 ('telecomunicaciones', 0.639496922492981),
 ('Wanzhou', 0.617560863494873),
 ('5G', 0.6000877022743225),
 ('Meng', 0.5700865387916565),
 ('Pekin', 0.5595781803131104),
 ('Ren', 0.5414323210716248),
 ('gigante_asiático', 0.5363011956214905),
 ('sanciones_estadounidenses', 0.5254279375076294),
 ('China', 0.5246084928512573)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["TVN"])

[('Televisión', 0.6506631374359131),
 ('directorio', 0.5564517974853516),
 ('Orrego', 0.48779889941215515),
 ('C5N', 0.47816959023475647),
 ('Panorama', 0.47550544142723083),
 ('Chilevisión', 0.46353694796562195),
 ('cuprífera', 0.4621235728263855),
 ('matinal', 0.45264914631843567),
 ('CHV', 0.4483409821987152),
 ('Mega', 0.44625139236450195)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["ultraderechista"])

[('excapitán_Ejército', 0.7039800882339478),
 ('ultraderecha', 0.7019724249839783),
 ('ultraderechista_Jair', 0.661270022392273),
 ('extrema_derecha', 0.6610966324806213),
 ('Finlandeses', 0.659309983253479),
 ('Verdaderos', 0.6557668447494507),
 ('antiinmigración', 0.6375353336334229),
 ('exmilitar', 0.636165201663971),
 ('Liga', 0.6324677467346191),
 ('Matteo_Salvini', 0.6320110559463501)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["tonto"])

[('estúpido', 0.7166074514389038),
 ('loco', 0.7140238285064697),
 ('¿Que', 0.7014246582984924),
 ('Dije', 0.6799631714820862),
 ('suena', 0.6779486536979675),
 ('pinta', 0.6729979515075684),
 ('haré', 0.6724047660827637),
 ('repugnante', 0.6709104180335999),
 ('dices', 0.6700376868247986),
 ('egoísta', 0.6675840616226196)]

### **Analogías**

Por otra parte, la analogía consiste en comparar 3 terminos mediante una operación del estilo:

$$palabra1 - palabra2 \approx palabra 3 - x$$

para encontrar relaciones entre estos.

Por ejemplo:

| palabra 1 (pos) |  palabra 2 (neg) |
|-----------------|------------------|
|  macri          | Argentina          |
| Brasil           |  x               |

In [ ]:
biobio_w2v.wv.most_similar(positive=["Macri", "Brasil"], negative=['Argentina'], topn=10)

[('Bolsonaro', 0.6208245158195496),
 ('ultraderechista_Jair', 0.5999900698661804),
 ('Jair_Bolsonaro', 0.5594528913497925),
 ('Michel_Temer', 0.5592767596244812),
 ('excapitán_Ejército', 0.556822657585144),
 ('Haddad', 0.532974123954773),
 ('Fernando_Haddad', 0.5272845029830933),
 ('Temer', 0.5139399766921997),
 ('Brasilia', 0.4959222972393036),
 ('Ultraderechista', 0.47941094636917114)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["Chile", "Huawei"], negative=['China'], topn=10)

[('Falabella', 0.47030484676361084),
 ('portabilidad', 0.4300999641418457),
 ('Multicaja', 0.4203709363937378),
 ('CMR', 0.41495582461357117),
 ('Sodimac', 0.4064332842826843),
 ('Valledor', 0.3986559212207794),
 ('Telefónica', 0.39562681317329407),
 ('Ciberseguridad', 0.39511701464653015),
 ('Subtel', 0.3942727744579315),
 ('retail', 0.39312079548835754)]

In [ ]:
biobio_w2v.wv.most_similar(positive=["perro", "tiburón"], negative=['gato'], topn=10)

[('canguro', 0.6266387104988098),
 ('domador', 0.6069827079772949),
 ('reptil', 0.6003487706184387),
 ('pez', 0.5977956652641296),
 ('jirafa', 0.5955661535263062),
 ('león', 0.5919601321220398),
 ('oso', 0.5904221534729004),
 ('macho', 0.5821169018745422),
 ('foca', 0.5812079310417175),
 ('serpiente', 0.5803099870681763)]

## **Word Embeddings como características para clasificar**


En esta sección, veremos como utilizar los word embeddings como característica para **clasificar nuevamente el tópico de las noticias de la radio biobio**.


Primero, obtendremos los datos y sus categorías y dejamos solo las primeras 20:

### Cargar el dataset

In [ ]:
dataset = dataset_r.copy(deep=True)

In [ ]:
# creamos una nueva columna titulo y contenido.
content = dataset['content']

# obtenemos las clases
subcategory = dataset.subcategory

# dejamos en el dataset solo contenido de la noticia y categoria
dataset = pd.DataFrame({'content': content, 'category': subcategory})

In [ ]:
NUM_SAMPLES = 250

categorias_seleccionadas = [
    'america-latina', 'eeuu', 'europa', 'chile', 'region-metropolitana',
    'region-del-bio-bio', 'negocios-y-empresas', 'region-de-los-lagos',
    'actualidad-economica', 'region-de-valparaiso', 'region-de-la-araucania',
    'curiosidades', 'asia', 'region-de-los-rios', 'entrevistas', 'debates',
    'mediooriente', 'viral', 'animales', 'tu-bolsillo'
]

# filtrar solo categorias seleccionadas
dataset = dataset[dataset['category'].isin(categorias_seleccionadas)]

# balancear clases
g = dataset.groupby('category')
dataset = pd.DataFrame(
    g.apply(lambda x: x.sample(NUM_SAMPLES).reset_index(drop=True))
).reset_index(drop=True)

Ahora, transformamos cada documento del dataset en el promedio de sus embeddings.


### Dividir el dataset en training y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset.content,
                                                    dataset.category,
                                                    test_size=0.33,
                                                    random_state=42)

Primero, crearemos el Transformer con el cual convertiremos el documento a vector. (puede que les sirva para la competencia...)


### Doc2vec

In [ ]:
class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    """ Transforma tweets a representaciones vectoriales usando algún modelo de Word Embeddings.
    """

    def __init__(self, model, aggregation_func):
        # extraemos los embeddings desde el objeto contenedor. ojo con esta parte.
        self.model = model.wv

        # indicamos la función de agregación (np.min, np.max, np.mean, np.sum, ...)
        self.aggregation_func = aggregation_func

    def simple_tokenizer(self, doc, lower=False):
        """Tokenizador. Elimina signos de puntuación, lleva las letras a minúscula(opcional) y
           separa el tweet por espacios.
        """
        if lower:
            doc.translate(str.maketrans('', '', string.punctuation)).lower().split()
        return doc.translate(str.maketrans('', '', string.punctuation)).split()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        doc_embeddings = []

        for doc in X:
            # tokenizamos el documento. Se llevan todos los tokens a minúscula.
            # ojo con esto, ya que puede que tokens con minúscula y mayúscula tengan
            # distintas representaciones
            tokens = self.simple_tokenizer(doc, lower = True)

            selected_wv = []
            for token in tokens:
                if token in self.model.index_to_key:
                    selected_wv.append(self.model[token])

            # si seleccionamos por lo menos un embedding para el tweet, lo agregamos y luego lo añadimos.
            if len(selected_wv) > 0:
                doc_embedding = self.aggregation_func(np.array(selected_wv), axis=0)
                doc_embeddings.append(doc_embedding)
            # si no, añadimos un vector de ceros que represente a ese documento.
            else:
                print('No pude encontrar ningún embedding en el tweet: {}. Agregando vector de ceros.'.format(doc))
                doc_embeddings.append(np.zeros(self.model.vector_size)) # la dimension del modelo

        return np.array(doc_embeddings)


### Definimos el pipeline


Usaremos la transformación que creamos antes mas una regresión logística.

In [ ]:
clf = LogisticRegression(max_iter=1000000)

doc2vec_mean = Doc2VecTransformer(biobio_w2v, np.mean)
doc2vec_sum = Doc2VecTransformer(biobio_w2v, np.sum)
doc2vec_max = Doc2VecTransformer(biobio_w2v, np.max)


pipeline = Pipeline([('doc2vec', doc2vec_sum), ('clf', clf)])

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('doc2vec',
                 Doc2VecTransformer(aggregation_func=<function sum at 0x7fbcb174ac10>,
                                    model=<gensim.models.keyedvectors.KeyedVectors object at 0x7fbca86be9d0>)),
                ('clf', LogisticRegression(max_iter=1000000))])

**Predecimos y evaluamos:**

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[43  0  0  0  9  0  0  1  1  0 22  0  0  0  0  3  0 12  0]
 [ 1 67  0  2  2  2  9  0  0  1  1  0  1  0  0  0  3  0  0]
 [ 0  1 55  7  0  3  2  0  0  0  2  0  0  0  0  0  0  0  5]
 [ 2  1  3 56  0  3  7  0  2  1  0  0  0  0  0  0  0  0  3]
 [10  3  0  0 53  0  0  5  0  0  6  2  1  1  5  3  5  6  1]
 [ 2  1  3  4  1 42  3  0  4  1  4  0  0  0  0  0  0  1 14]
 [ 1  2  1  4  0  3 66  0  2  5  0  0  0  0  0  0  0  0  2]
 [ 3  0  0  0  7  1  0 53  0  0  1  1  0  2  2  0  2  2  1]
 [ 0  5  1  7  0  2  4  1 65  0  1  0  0  0  0  0  1  0  1]
 [ 0  1  0  2  0  0  8  0  6 66  0  0  0  0  0  0  0  0  2]
 [19  0  0  0  3  1  0  2  0  0 35  0  2  0  0  3  1 11  1]
 [ 1  0  0  0  1  0  0  1  0  0  2 61  4  5  1  7  3  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  2  1 59  8  4  1  1  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  1  6  5 62  3  2  0  0  0]
 [ 2  1  1  1  2  0  0  3  0  0  0  2  4  0 40  9 11  0  0]
 [ 0  1  0  0  2  0  0  1  0  0  1  3  1  4  3 59  4  0  0]
 [ 1  0  1  0  3  1  0  1  0  0  1  2  2

In [ ]:
print(classification_report(y_test, y_pred))

                        precision    recall  f1-score   support

  actualidad-economica       0.44      0.47      0.46        91
        america-latina       0.81      0.75      0.78        89
              animales       0.73      0.73      0.73        75
                  asia       0.67      0.72      0.69        78
                 chile       0.57      0.52      0.55       101
          curiosidades       0.59      0.53      0.56        80
                  eeuu       0.64      0.77      0.70        86
           entrevistas       0.76      0.71      0.73        75
                europa       0.81      0.74      0.77        88
          mediooriente       0.89      0.78      0.83        85
   negocios-y-empresas       0.38      0.45      0.41        78
region-de-la-araucania       0.78      0.71      0.74        86
   region-de-los-lagos       0.75      0.77      0.76        77
    region-de-los-rios       0.74      0.78      0.76        80
  region-de-valparaiso       0.61      

In [ ]:
pipeline.predict(
    [("Alguna noticia..")])

array(['region-del-bio-bio'], dtype=object)

## **Usandolo con BoW**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset.content,
                                                    dataset.category,
                                                    test_size=0.33,
                                                    random_state=42)

In [ ]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))

# Definimos el clasificador que usaremos.
clf_2 = LogisticRegression(max_iter=10000)

# Definimos el pipeline
pipeline_2 = Pipeline([('features',
                        FeatureUnion([('bow', CountVectorizer()),
                                      ('doc2vec', doc2vec_sum)])), ('clf', clf)])

In [ ]:
pipeline_2.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('bow', CountVectorizer()),
                                                ('doc2vec',
                                                 Doc2VecTransformer(aggregation_func=<function sum at 0x7fbcb174ac10>,
                                                                    model=<gensim.models.keyedvectors.KeyedVectors object at 0x7fbca86be9d0>))])),
                ('clf', LogisticRegression(max_iter=1000000))])

In [ ]:
y_pred_2 = pipeline_2.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred_2)
print(conf_matrix)

[[55  0  0  0  8  0  1  1  1  0 15  0  0  0  0  2  0  8  0]
 [ 1 68  0  1  2  2  6  0  3  0  1  0  0  0  0  1  3  0  1]
 [ 0  3 62  3  0  2  0  0  0  0  0  0  0  0  0  0  0  0  5]
 [ 1  1  1 61  0  3  4  0  1  2  0  0  0  0  0  0  0  0  4]
 [ 5  4  1  0 61  0  0  5  1  0  2  3  2  0  4  2  5  6  0]
 [ 0  1  4  3  0 48  3  0  2  0  3  0  0  0  0  0  0  1 15]
 [ 1  2  1  5  0  2 67  0  2  5  0  0  0  0  0  0  0  0  1]
 [ 3  0  0  0  2  0  0 64  0  0  1  1  0  0  1  0  0  3  0]
 [ 1  2  1  3  0  2  4  0 73  1  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  3  0  1  5  0  3 72  0  0  0  0  0  0  0  0  1]
 [11  0  0  0  2  2  0  2  0  0 49  1  1  0  0  2  1  7  0]
 [ 0  0  0  0  1  0  0  1  0  0  0 68  3  6  2  5  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  2  1 67  4  1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  3 71  4  1  0  0  0]
 [ 1  0  1  0  1  0  0  1  0  0  1  0  5  0 59  5  2  0  0]
 [ 0  0  0  0  1  1  0  0  0  0  0  0  1  2  2 70  1  0  1]
 [ 1  0  0  0  6  1  0  1  0  0  2  1  2

In [ ]:
print(classification_report(y_test, y_pred_2))

                        precision    recall  f1-score   support

  actualidad-economica       0.62      0.60      0.61        91
        america-latina       0.84      0.76      0.80        89
              animales       0.78      0.83      0.80        75
                  asia       0.76      0.78      0.77        78
                 chile       0.66      0.60      0.63       101
          curiosidades       0.62      0.60      0.61        80
                  eeuu       0.74      0.78      0.76        86
           entrevistas       0.85      0.85      0.85        75
                europa       0.84      0.83      0.83        88
          mediooriente       0.90      0.85      0.87        85
   negocios-y-empresas       0.55      0.63      0.59        78
region-de-la-araucania       0.89      0.79      0.84        86
   region-de-los-lagos       0.80      0.87      0.83        77
    region-de-los-rios       0.86      0.89      0.87        80
  region-de-valparaiso       0.77      

### Propuesto...

- Usar su modelo de embeddings favorito para ver si mejora la clasificación:
    
 - Fast y word2vec en español, [cortesía](https://github.com/dccuchile/spanish-word-embeddings) de los grandes del DCC
 - [Conceptnet](https://github.com/commonsense/conceptnet-numberbatch)


- Visualizar los documentos usando `doc2vec`

## Clasificación de texto usando CNN + Embeddings

Para este caso vamos a trabajar con un dataset de noticias, el cual es fácilmente descargable con la librería y da muchos mejores resultados (ya que los anteriores estaban ahí nomas).

In [ ]:
# Instalamos portalocker para acceder a los datasets de Pytorch
!pip install portalocker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# https://pytorch.org/text/stable/datasets.html#ag-news
import os
import torch
from random import choice
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

os.makedirs("data", exist_ok=True)
train_dataset, test_dataset = AG_NEWS(root="data", split=('train', 'test'))
train_list = list(train_dataset)
test_list = list(test_dataset)

# Informacion relevante del dataset
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(tokenizer(x[1]) for x in train_list)

vocab.set_default_index(0)
vocab.insert_token('<pad>', 1)

stoi = vocab.get_stoi()

num_classes = 4



Luego, creamos una red no tan profunda pero bien competente para nuestra tarea:

In [ ]:
import torch.nn as nn
from itertools import zip_longest
import torch.nn.functional as F

class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, num_classes=10,
                 cnn_pool_channels=24, cnn_kernel_size=3):

        # Inicializamos la clase padre
        super().__init__()

        # Creamos la capa de embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Creamos la capa de convolución
        # `in_channels`: Es el número de canales de entrada de la convolución. En este caso, como estamos trabajando con texto, sólo tenemos un canal, por lo que `in_channels=1`.
        # `out_channels`: Es el número de canales de salida de la convolución. Especifica la cantidad de filtros que se aplicarán a la entrada. En este caso, queremos generar `cnn_pool_channels` canales de salida, por lo que `out_channels=cnn_pool_channels`.
        # `kernel_size`: Es el tamaño del kernel de la convolución. En este caso, estamos usando un kernel de tamaño `cnn_kernel_size * embed_dim`, donde `embed_dim` es la dimensión de los vectores de embedding. Esto significa que cada filtro de la convolución cubrirá `cnn_kernel_size` palabras (o tokens) en una dimensión y `embed_dim` en la otra.
        # `stride`: Es el desplazamiento que se aplica a la entrada de la convolución. En este caso, estamos desplazando la entrada `embed_dim` unidades en cada paso. Esto significa que se aplicarán filtros a cada palabra (o token) de la entrada.
        self.conv = nn.Conv1d(
            in_channels=1,
            out_channels=cnn_pool_channels,
            kernel_size=cnn_kernel_size * embed_dim,
            stride=embed_dim,
        )

        # Calculamos el tamaño de entrada de la capa lineal
        fc_in_size = cnn_pool_channels

        # Creamos la capa lineal
        self.fc = nn.Linear(fc_in_size, num_classes)

        # Inicializamos los pesos de las capas
        self.init_weights()

    def init_weights(self):
        # Definimos el rango de los valores iniciales de los pesos
        initrange = 0.5

        # Inicializamos los pesos de la capa de embedding
        self.embedding.weight.data.uniform_(-initrange, initrange)

        # Inicializamos los pesos de la capa lineal
        self.fc.weight.data.uniform_(-initrange, initrange)

        # Inicializamos los sesgos de la capa lineal en cero
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):

        # Preparamos el input de la capa de embeddings a partir de text y offsets
        text = torch.tensor(
            list(
                zip(
                    *zip_longest(
                        *([text[o:offsets[i+1]] for i, o in enumerate(offsets[:-1])] + [text[offsets[-1]:len(texts)]]),
                        fillvalue=vocab["<pad>"]
                    )
                )
            )
        ).to(text.device)

        # Obtenemos la representación de la frase a partir de la capa de embedding
        h = self.embedding(text)

        # Aplicamos la capa de convolución
        h = h.view(h.size(0), 1, -1)
        h = torch.relu(self.conv(h))
        h = h.mean(dim=2)

        # Obtenemos el resultado final a partir de la capa lineal
        output = self.fc(h)

        # Aplicamos la función de activación log-softmax
        return F.log_softmax(output, dim=1)

Finalmente, generamos la función para cargar por batch y luego entrenamos directamente.

In [ ]:
import sys
from torch.optim import SGD, lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable

def generate_batch(batch):
  label = torch.tensor([entry[0]-1 for entry in batch])
  texts = [tokenizer(entry[1]) for entry in batch]
  offsets = [0] + [len(text) for text in texts]
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  big_text = torch.cat([torch.tensor([vocab[t] if t in stoi else 0 for t in text]) for text in texts])
  #big_text = torch.cat([torch.tensor([vocab.stoi[t] for t in text]) for text in texts])

  return big_text, offsets, label

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

BATCH_SIZE = 256
NUM_EPOCHS = 20
TEST_BATCH_SIZE = BATCH_SIZE * 5
LR = 1e-1

model = CNNClassifier(len(vocab), num_classes=num_classes).to(device)
optimizer = SGD(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: .9 ** (epoch // 10)])

split_size = {'train': len(train_list), 'test': len(test_list)}

# train_dataset, test_dataset = AG_NEWS(root="data")
for epoch in range(1, NUM_EPOCHS):
  train_loader = DataLoader(train_list, batch_size=BATCH_SIZE, collate_fn=generate_batch)
  test_loader = DataLoader(test_list, batch_size=TEST_BATCH_SIZE, collate_fn=generate_batch)
  loaders = {'train': train_loader, 'test': test_loader}
  for phase in ['train', 'test']:
    if phase == 'train':
      model.train()
    else:
      model.eval()

    total_acc, total_loss = 0, 0
    for i, (texts, offsets, cls) in enumerate(loaders[phase]):
      texts = texts.to(device)
      offsets = offsets.to(device)
      cls = cls.to(device)

      optimizer.zero_grad()
      with torch.set_grad_enabled(phase == 'train'):
        output = model(texts, offsets)
        loss = criterion(output, cls)
        total_loss += loss.item()
        if phase == 'train':
          loss.backward()
          optimizer.step()

      acc = (output.argmax(1) == cls).sum().item()
      total_acc += acc

      sys.stdout.write('\rEpoch: {0:03d}\t Phase: {1} Iter: {2:03d}/{3:03d}\t iter-Acc: {4:.3f}%\t iter-Loss: {5:.3f}'.format(epoch, phase, i+1, len(loaders[phase]), acc/len(offsets)*100, loss.item()))

    if phase == 'train':
      scheduler.step()
    print('\n {0}\tAvg. Acc: {1:.3f}%\t Avg. Loss: {2:.3f}'.format(phase, total_acc/split_size[phase]*100, total_loss/split_size[phase]))

cuda
Epoch: 001	 Phase: train Iter: 469/469	 iter-Acc: 23.958%	 iter-Loss: 1.377
 train	Avg. Acc: 26.982%	 Avg. Loss: 0.005
Epoch: 001	 Phase: test Iter: 006/006	 iter-Acc: 26.083%	 iter-Loss: 1.383
 test	Avg. Acc: 25.118%	 Avg. Loss: 0.001
Epoch: 002	 Phase: train Iter: 469/469	 iter-Acc: 26.562%	 iter-Loss: 1.369
 train	Avg. Acc: 29.483%	 Avg. Loss: 0.005
Epoch: 002	 Phase: test Iter: 006/006	 iter-Acc: 29.667%	 iter-Loss: 1.380
 test	Avg. Acc: 29.329%	 Avg. Loss: 0.001
Epoch: 003	 Phase: train Iter: 469/469	 iter-Acc: 31.771%	 iter-Loss: 1.357
 train	Avg. Acc: 31.831%	 Avg. Loss: 0.005
Epoch: 003	 Phase: test Iter: 006/006	 iter-Acc: 33.583%	 iter-Loss: 1.375
 test	Avg. Acc: 33.000%	 Avg. Loss: 0.001
Epoch: 004	 Phase: train Iter: 469/469	 iter-Acc: 39.583%	 iter-Loss: 1.339
 train	Avg. Acc: 34.222%	 Avg. Loss: 0.005
Epoch: 004	 Phase: test Iter: 006/006	 iter-Acc: 35.500%	 iter-Loss: 1.367
 test	Avg. Acc: 35.013%	 Avg. Loss: 0.001
Epoch: 005	 Phase: train Iter: 469/469	 iter-Acc: 4